<a href="https://colab.research.google.com/github/sofiadelamora/HumanTMexicoNetwork/blob/main/MistralFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install required packages

In [1]:
!pip install -U transformers huggingface_hub

  Using cached transformers-4.41.2-py3-none-any.whl (9.1 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 46.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2


In [2]:
!pip install -q accelerate==0.28.0 peft==0.4.0 bitsandbytes==0.41.3 trl==0.4.7 ipywidgets==7.7.1 transformers==4.38.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 18.5 MB/s eta 0:00:00


# Imports

In [3]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from peft.utils import prepare_model_for_kbit_training
from trl import SFTTrainer
from google.colab import files, userdata
from datasets import Dataset
import pandas as pd
import json
import re
# from huggingface_hub import notebook_login

In [4]:
hf_auth = userdata.get('HF_TOKEN_2')

# Load model

In [5]:
# Model to train
model_name = "mistralai/Mistral-7B-Instruct-v0.3"

In [6]:
# Conversation dataset to use
dataset_path = "/content/conversations.json"

In [7]:
# Fine-tuned model name
new_model = "Mistral-7b-instruct-v0.3-finetuned"

## QLoRA parameters

In [8]:
# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

## Bitsandbytes parameters

In [9]:
# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

## TrainingArguments parameters

In [10]:
# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer t to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

## SFT parameters

In [11]:
# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

# Load finetune material

In [12]:
def validate_json_line(line):
    try:
        return json.loads(line)
    except json.JSONDecodeError:
        print(f"Invalid JSON line: {line}")
        return None

In [14]:
data = []
with open(dataset_path, 'r', encoding='utf-8') as f:
    for line in f:
        json_obj = validate_json_line(line)
        if json_obj is not None:
            data.append(json_obj)

# Verify loaded data
print(f"Loaded {len(data)} records")

# Convert JSON data to a Pandas DataFrame
df = pd.DataFrame(data)

# Convert DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Shuffle and select samples
dataset = dataset.shuffle(seed=42).select(range(min(len(dataset), 5000)))

Invalid JSON line: {"text": "<s>[INST] Quick question, I got my blood draw [TIME] to recheck my thyroid. I finished my last active birth control [TIME]. Do I open a new pack and keep taking active pills? If my bloodwork was good, I didn't know if we would jump into a transfer timeline. [/INST] We will have you get another pack to continue your ACTIVE ONLY pills! Fingers crossed your level comes back good to proceed with transfer!!\n\n[INST] I have another pack, so I'll just keep taking the active! \ud83e\udd1e\ud83e\udd1e\ud83e\udd1e I hope it comes back good too!! Thanks! [/INST] I will let you know!!\n\n[/INST] Hi Kelly...your [ORG] came back at 1.55 :) Dr. [PERSON] reviewed your results and would like you to remain on your same dose of [ORG]. Let us know when you need an additional refill. In terms of a transfer date...how does this sound: Last birth control 2/20; BASE ultrasound 2/25; LINING check 3/11; [ORG]. Would this work for you? :)\n\n[INST] Oh good! Yes, that works. Yay! [IN

In [15]:
# Split the dataset into train and test sets
train_test_split = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

# Verify the splits
print(f"Train dataset length: {len(train_dataset)}")
print(f"Test dataset length: {len(test_dataset)}")

Train dataset length: 4000
Test dataset length: 1000


In [16]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [17]:
# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

Your GPU supports bfloat16: accelerate training with bf16=True


In [18]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map,
    use_auth_token=hf_auth,
)
model.config.use_cache = False
model.config.pretraining_tp = 1

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:466: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [19]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, token=hf_auth,)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:720: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [20]:
# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
)

In [21]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
)

In [22]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
# Train model
trainer.train()

Step,Training Loss
25,2.185100
50,1.698400
75,1.621600
100,1.224100
125,1.397800
150,1.224300
175,1.472600
200,1.187200
225,1.385300
250,1.123300


In [ ]:
# Save trained model
trainer.model.save_pretrained(new_model)

# Store finetuned model

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)

In [ ]:
prompt = "Do you have available apointments?"
# prompt = 'What medication can I take for the pain?'

In [ ]:
# Run text generation pipeline with the base model
base_pipe = pipeline(task="text-generation", model=base_model, tokenizer=tokenizer, max_length=200)
base_result = base_pipe(f"<s>[INST] {prompt} [/INST]")
generated_text = base_result[0]['generated_text']

In [ ]:
matchs = re.search(r'\[\/INST\](.*?)\[INST\]', generated_text, re.DOTALL)

if matchs:
    extracted_text = matchs.group(1).strip()
else:
    extracted_text = generated_text

# Print the extracted response
print("Base model response:")
print(extracted_text)

In [ ]:
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Visual analisys

In [ ]:
%load_ext tensorboard
%tensorboard --logdir results/runs

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

In [ ]:
# Run text generation pipeline with the fine-tuned model
fine_tuned_pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
fine_tuned_result = fine_tuned_pipe(f"<s>[INST] {prompt} [/INST]")
generated_text = fine_tuned_result[0]['generated_text']

# Print the extracted response
print("Fine-tuned model response:")
print(generated_text)
